In [1]:
import pandas as pd

# Cargar el archivo CSV
data = pd.read_csv('data.csv')
data = pd.read_csv('data.csv', delimiter=',') 

In [2]:
import pandas as pd

data.loc[(data['Sector'] == 'DISEÑO') & (data['Etapa'] == 'MODELOS'), 'Etapa'] = 'MODELO_D'
data.loc[(data['Sector'] == 'ESCANEADO') & (data['Etapa'] == 'MODELOS'), 'Etapa'] = 'MODELO_E'


# Convertir las columnas a tipo datetime para asegurar un correcto manejo de fechas
data['Início'] = pd.to_datetime(data['Início'])
data['Fecha prevista'] = pd.to_datetime(data['Fecha prevista'])
data['Última Finalización'] = pd.to_datetime(data['Última Finalización'])

# Pivotar los datos para obtener una columna por cada Etapa con el valor de 'Empleado'
pivot_data_empleado = data.pivot_table(index=['Nº', 'Ctd', 'Servicio', 'Fecha prevista', 'Última Finalización'], 
                                        columns='Etapa', 
                                        values='Empleado', 
                                        aggfunc='first', 
                                        fill_value=0)

# Restablecer el índice para obtener un DataFrame normal
pivot_data_empleado = pivot_data_empleado.reset_index()

# Agrupar por las columnas relevantes y tomar el máximo para cada etapa
grouped_data = pivot_data_empleado.groupby(['Nº', 'Ctd', 'Servicio'], as_index=False).agg(lambda x: x.max())

# Calcular la primera fecha de 'Início' para cada combinación de 'Nº', 'Servicio' y 'Ctd'
fecha_inicio = data.groupby(['Nº', 'Servicio', 'Ctd'])['Início'].min().reset_index() 
fecha_inicio.rename(columns={'Início': 'Fecha Inicio'}, inplace=True)

# Obtener el máximo de las columnas 'Fecha prevista' y 'Última Finalización' por 'Nº', 'Servicio' y 'Ctd'
max_dates = data.groupby(['Nº', 'Servicio', 'Ctd']).agg({'Fecha prevista': 'max', 'Última Finalización': 'max'}).reset_index()

# Fusionar las fechas máximas con los datos agrupados
max_data = pd.merge(grouped_data, fecha_inicio, on=['Nº', 'Servicio', 'Ctd'], how='left')
max_data = pd.merge(max_data, max_dates, on=['Nº', 'Servicio', 'Ctd'], how='left')

# Eliminar duplicados
max_data = max_data.drop_duplicates()

# Guardar el nuevo DataFrame en un archivo CSV
max_data.to_csv('base.csv', index=False)

In [3]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('base.csv')

# Eliminar las últimas dos columnas
df = df.iloc[:, :-2]

df.to_excel('base.xlsx', index=False, engine='openpyxl')
